## Setting up

### Which networks do we want to analyse and where are they?

In [1]:
webnames = ["Premonsoon","Postmonsoon","Monsoon"];
locations = ["../data/" ];

# Do we want to analyse webs with or without NCP?
suffix = ""; #"_NCP"

#webnames = ["Final_BSQ", "Final_CHB", "Final_CPP", "Final_CSF","Final_CSM",
#    "Final_EMB","Final_EPB","Final_EWB","Final_LRL","Final_MPC","Final_MRM",
#    "Final_PCR","Final_PRV","Final_STM","Final_YTH"];
#locations = ["../data/Lauras/"];

## Functions

In [2]:
#Loading packages

using Pkg
#Pkg.add("EcologicalNetworks")

#import Pkg; Pkg.add("CSV"); Pkg.add("DataFrames")
using CSV
using DataFrames
using EcologicalNetworks
using EcologicalNetworksPlots



In [3]:
# Setting up functions

#This function opens and reads all the files
function openwebs(locations,webnames,suffix)
    # Check that location is either the same for everything, or that we've got one location per webname
    if length(locations) == 1
        locations = repeat(locations, length(webnames))
    elseif length(locations) != length(webnames)
        error("Need only one location, or a vector of equal length to webnames")
    end
    
    dfs = []

    for i in 1:length(webnames)
        filename = locations[i] * webnames[i] * "_adjacency_matrix" * suffix * ".csv"

        df = CSV.read(filename,DataFrame)

        # Append the df to the list
        push!(dfs, df)

    end

    return dfs

end

openwebs (generic function with 1 method)

In [4]:
# This extracts the species names from the files
function getspecies(webs)
    Ss = []

    for web in webs
        # Get the first column which has the names of all the species
        S = web.Column1
        
        # Force it to be strings because integers don't work
        S = string.(S)
        
        # Append S to the list
        push!(Ss, S)

    end

    return Ss

end

getspecies (generic function with 1 method)

In [5]:
# Convert webs to boolean array
function convertWebsToBoolArray(webs)
    As = []

    for web in webs
        ## Remove the column with the names
        A = Array(web[:,2:end])
        A = transpose(A)

        # Convert to boolean
        A = isone.(A)

        # Append A to the list
        push!(As, A)

    end

    return As

end

convertWebsToBoolArray (generic function with 1 method)

In [6]:
# Convert the output into a dataframe with a column for species, value of the metric, web name, and metric
function dicts_to_df(dicts, webnames, metric)
    
    dfs_wide = DataFrame.(dicts)
    dfs_long = [stack(df, names(df)) for df in dfs_wide]

    
    for i in 1:size(webnames)[1]
        dfs_long[i][!, :webname] .= webnames[i]
        rename!(dfs_long[i], names(dfs_long[i])[1] => "species")
        rename!(dfs_long[i], names(dfs_long[i])[2] => metric)
    end   
    
    return dfs_long
end

dicts_to_df (generic function with 1 method)

In [7]:
function motifs_to_csv(motif_list, webnames)

    # Create a vector to hold the column labels for the three integer columns
    col_labels = ["sp1", "sp2", "sp3"]

    # Create a vector to hold the data for each column
    sp1_data = Vector{String}()
    sp2_data = Vector{String}()
    sp3_data = Vector{String}()
    web_data = Vector{String}()
    motif_data = Vector{Symbol}()
    
    # Loop over each element of the matrix and extract the data to the appropriate vectors
    motif_names = keys(unipartitemotifs())

    for i in 1:size(motif_lists, 1)
        for j in 1:size(motif_lists, 2)
            for k in 1:size(motif_lists[i, j])[1]
                sp1, sp2, sp3 = motif_lists[i, j][k][1]
                push!(sp1_data, Ss[j][sp1])
                push!(sp2_data, Ss[j][sp2])
                push!(sp3_data, Ss[j][sp3])
                push!(web_data, webnames[j])
                push!(motif_data, motif_names[i])
            end
        end
    end
    
    # Create the dataframe using the extracted data and column labels
    motif_df = DataFrame(
        sp1 = sp1_data,
        sp2 = sp2_data,
        sp3 = sp3_data,
        web = web_data,
        motif = motif_data
    )
    
    return motif_df
end

motifs_to_csv (generic function with 1 method)

## Our data

In [8]:
webs = openwebs(locations, webnames, suffix);

In [9]:
# Get lists of species
Ss = getspecies(webs);

In [10]:
# Convert networks to boolean arrays, then to unipartite networks
As = convertWebsToBoolArray(webs);
Ns = [UnipartiteNetwork(As[j],Ss[j]) for j in 1:length(As)];


## Network level analysis

In [11]:
# How many links does each network have?
links_byweb = [links(N) for N in Ns];

In [12]:
size_byweb = [size(S)[1] for S in Ss];

In [13]:
# What's the connectance of each network?
C_byweb = [connectance(N) for N in Ns];


In [14]:
nestedness_byweb = [ρ(N) for N in Ns];

In [15]:
web_metrics_df = DataFrame(web = webnames, size = size_byweb, links = links_byweb, C = C_byweb, nestedness = nestedness_byweb);

In [16]:
filename = locations[1] * "processed/network_metrics" * suffix * ".csv" 
CSV.write(filename, web_metrics_df, writeheader=true);


## Species level metrics

In [17]:
# Calculate degree for each species in each web
degrees_byweb = [EcologicalNetworks.degree(N) for N in Ns];
degree_dfs = dicts_to_df(degrees_byweb, webnames, "degree");

outdegrees_byweb = [EcologicalNetworks.degree(N, dims = 1) for N in Ns];
outdegree_dfs = dicts_to_df(outdegrees_byweb, webnames, "outdegree");

indegrees_byweb = [EcologicalNetworks.degree(N, dims = 2) for N in Ns];
indegree_dfs = dicts_to_df(indegrees_byweb, webnames, "indegree");

TL_byweb = [trophic_level(N) for N in Ns];
TL_dfs = dicts_to_df(TL_byweb, webnames, "TL");

omnivory_byweb = [omnivory(N) for N in Ns];
omnivory_dfs = dicts_to_df(omnivory_byweb, webnames, "omnivory");


In [18]:
# Join the individual metric dataframes
dfs = outerjoin.(TL_dfs, degree_dfs,    on = [:species, :webname]);
dfs = outerjoin.(dfs,    indegree_dfs,  on = [:species, :webname]);
dfs = outerjoin.(dfs,    outdegree_dfs, on = [:species, :webname]);
dfs = outerjoin.(dfs,    omnivory_dfs,  on = [:species, :webname]);

In [19]:
# Put it all into one big dataframe for printing
df_long = vcat([hcat(df, DataFrame(web = repeat([webnames[i]], nrow(df)))) for (i, df) in enumerate(dfs)]...);

filename = locations[1] * "processed/species_metrics" * suffix * ".csv" 
CSV.write(filename, df_long, writeheader=true);


## Motifs


In [20]:
# Get all the motif tuples for each motif and each web
motif_lists = [find_motif(N,m) for m in unipartitemotifs(), N in Ns];

In [21]:
# Convert it all into one big dataframe
motif_df = motifs_to_csv(motif_lists, webnames);

In [22]:
filename = locations[1] * "processed/motif_lists" * suffix * ".csv" 
CSV.write(filename, motif_df, writeheader=true);

# Old code to be integrated